In [2]:
# Import all sort of things
import numpy as np
import scipy.linalg as splin
from scipy.integrate import LSODA, solve_ivp
from consts import *
from formulae import *
from potential import *


In [3]:
mE = 5.972e24
mM = 7.342e22
dEM = 384400
mu = mM/(mE+mM)
eps = 1e-14


In [4]:
# L1
xL1 = 0.8369
yL1 = 0 
XL1 = [xL1, yL1]

In [5]:
alpha0 = 50/384400 # initial guess for alpha

In [6]:
# Compute A at XL
A_xl = A(XL1[:2], mu)

In [7]:
L, V = splin.eig(A_xl)

print(V)

[[ 2.93321242e-01+0.00000000e+00j  2.93321242e-01+0.00000000e+00j
   1.05809301e-01-1.58426437e-17j  1.05809301e-01+1.58426437e-17j]
 [-1.35014101e-01+0.00000000e+00j  1.35014101e-01+0.00000000e+00j
   1.58471957e-18+3.79384181e-01j  1.58471957e-18-3.79384181e-01j]
 [ 8.59728492e-01+0.00000000e+00j -8.59728492e-01+0.00000000e+00j
  -1.07891337e-16+2.46930232e-01j -1.07891337e-16-2.46930232e-01j]
 [-3.95728140e-01+0.00000000e+00j -3.95728140e-01+0.00000000e+00j
  -8.85379859e-01+0.00000000e+00j -8.85379859e-01-0.00000000e+00j]]


In [9]:


def f_lyp (t_span: np.ndarray, om_ly, alpha0, V):
    # t_span is the time span [0, T]
    # om_ly is the angular velocity
    # alpha0 is the initial guess for alpha
    # V is the eigenvector corresponding to the eigenvalue with positive imaginary part
    V_real = V.real
    V_imag = V.imag
    X = np.zeros((2, len(t_span)))  
    for i, t in enumerate(t_span):
        X[:, i] = XL1 + alpha0*(V_real*np.cos(om_ly*t) - V_imag*np.sin(om_ly*t))
    return X





In [11]:
print("Eigenvalues at L1: ", L)
# Print formatted eigenvectors
print("Eigenvectors at L1: Before reformating") 
for i in range(4):
    print(V[:,i])

V.real[np.abs(V.real) <= eps] = 0
V.imag[np.abs(V.imag) <= eps] = 0
print("Eigenvectors at L1: After reformating") 
for i in range(4):
    print(V[:,i])



# Take the complex eigenvectors
L_compl = [(idx, x) for idx, x in enumerate(L) if isinstance(x, complex) and x.imag > eps]
# Print the complex eigenvectors
print("Complex eigenvectors at L1: ", L_compl)

# Take the eigenvector corresponding to the complex eigenvalue
v = V[:,L_compl[0][0]]

#Print real and imaginary parts of the eigenvector

v_real = np.zeros(4)
v_imag = np.zeros(4)
for j in range(4):
    if abs(v[j].real) > eps:
        v_real[j] = v[j].real
    if abs(v[j].imag) > eps:
        v_imag[j] = v[j].imag
# Recompose the eigenvector
v = v_real + 1j*v_imag
print("Real part of the eigenvector: ", v_real)
print("Imaginary part of the eigenvector: ", v.imag)
print("Eigenvector at L1: ", v)
# Take the first two components, associated with the position
v = v[:2]

# The angular velocity is the imaginary part of the eigenvalue
om_ly = L_compl[0][1].imag
# Print the angular velocity
print("Angular velocity: ", om_ly)

# Time span
t_span = np.linspace(0, 2*np.pi/om_ly, 100) # 1 period

sol = f_lyp(t_span, om_ly, alpha0, v)









Eigenvalues at L1:  [ 2.9310134+0.j         -2.9310134+0.j          0.       +2.33372898j
  0.       -2.33372898j]
Eigenvectors at L1: Before reformating
[ 0.29332124+0.j -0.1350141 +0.j  0.85972849+0.j -0.39572814+0.j]
[ 0.29332124+0.j  0.1350141 +0.j -0.85972849+0.j -0.39572814+0.j]
[ 1.05809301e-01-1.58426437e-17j  1.58471957e-18+3.79384181e-01j
 -1.07891337e-16+2.46930232e-01j -8.85379859e-01+0.00000000e+00j]
[ 1.05809301e-01+1.58426437e-17j  1.58471957e-18-3.79384181e-01j
 -1.07891337e-16-2.46930232e-01j -8.85379859e-01-0.00000000e+00j]
Eigenvectors at L1: After reformating
[ 0.29332124+0.j -0.1350141 +0.j  0.85972849+0.j -0.39572814+0.j]
[ 0.29332124+0.j  0.1350141 +0.j -0.85972849+0.j -0.39572814+0.j]
[ 0.1058093 +0.j          0.        +0.37938418j  0.        +0.24693023j
 -0.88537986+0.j        ]
[ 0.1058093 +0.j          0.        -0.37938418j  0.        -0.24693023j
 -0.88537986+0.j        ]
Complex eigenvectors at L1:  [(2, np.complex128(2.333728983254233j))]
Real part of t

In [ ]:
# Plotting cell
# Plot the Lagrange point and the Moon position
import matplotlib.pyplot as plt

t_span = np.linspace(0, 2*np.pi/om_ly, 100)

sol = f_lyp(t_span, om_ly, alpha0, v)

# Find from this solution the one with a crossing with the x-axis

# Plot the Lagrange point
plt.plot(XL1[0], XL1[1], 'ro', label='L1')
# Plot the solution
plt.plot(sol[0,:], sol[1,:], label='Lyapunov orbit')



# From X_lyap take the solution that crosses the x-axis


plt.xlabel('x')
plt.ylabel('y')

plt.legend()
plt.show()



